In [95]:
%pylab inline

ROW = "ABCDEFGHI"
COL = "123456789"

line = "003020600900305001001806400008102900700000008006708200002609500800203009005010300"

board = {ROW[r] + COL[c]: int(line[9*r+c])
                  for r in range(9) for c in range(9)}

print(board)

Populating the interactive namespace from numpy and matplotlib
{'A1': 0, 'A2': 0, 'A3': 3, 'A4': 0, 'A5': 2, 'A6': 0, 'A7': 6, 'A8': 0, 'A9': 0, 'B1': 9, 'B2': 0, 'B3': 0, 'B4': 3, 'B5': 0, 'B6': 5, 'B7': 0, 'B8': 0, 'B9': 1, 'C1': 0, 'C2': 0, 'C3': 1, 'C4': 8, 'C5': 0, 'C6': 6, 'C7': 4, 'C8': 0, 'C9': 0, 'D1': 0, 'D2': 0, 'D3': 8, 'D4': 1, 'D5': 0, 'D6': 2, 'D7': 9, 'D8': 0, 'D9': 0, 'E1': 7, 'E2': 0, 'E3': 0, 'E4': 0, 'E5': 0, 'E6': 0, 'E7': 0, 'E8': 0, 'E9': 8, 'F1': 0, 'F2': 0, 'F3': 6, 'F4': 7, 'F5': 0, 'F6': 8, 'F7': 2, 'F8': 0, 'F9': 0, 'G1': 0, 'G2': 0, 'G3': 2, 'G4': 6, 'G5': 0, 'G6': 9, 'G7': 5, 'G8': 0, 'G9': 0, 'H1': 8, 'H2': 0, 'H3': 0, 'H4': 2, 'H5': 0, 'H6': 3, 'H7': 0, 'H8': 0, 'H9': 9, 'I1': 0, 'I2': 0, 'I3': 5, 'I4': 0, 'I5': 1, 'I6': 0, 'I7': 3, 'I8': 0, 'I9': 0}


In [96]:
def get_box(key):
    for r_idx, r in enumerate(ROW):
        for c_idx, c in enumerate(COL):
            if (key[0] == r) and (key[-1] == c):    
                return boxes[r_idx][c_idx]

def check_cols(n, col, board):
    for key in board.keys():
        if key[-1] == col:
            if board[key] == n:
                return False
    return True

def check_rows(n, row, board):
    for key in board.keys():
        if key[0] == row:
            if board[key] == n:
                return False
    return True

def check_box(n, box):
    global box_to_vals
    if n in box_to_vals[box]:
        return False
    return True
    

In [97]:
boxes = [[(i//3,j//3) for i in range(9)] for j in range(9)]
box_to_vals = dict()
for key in board.keys():
    val = board[key]
    box = get_box(key)

    if box_to_vals.get(box):
        box_to_vals[box].append(val)
    else:
        box_to_vals[box] = [val]

In [99]:
def is_legal(n, coord, board):
    return (check_cols(n, coord[-1], board) and check_rows(n, coord[0], board) and check_box(n, get_box(coord)))

In [129]:
def solve(board):
    b = board.copy()
    if not 0 in b.values():
        return b
    for r in ROW:
        for c in COL:
            if b[r+c] == 0:
                for n in range(1,10):
                    if is_legal(n, r+c, b):
                        b[r+c] = n
                        result = solve(b)
                        if result != 'failure':
                            return result
                        b[r+c] = 0
                return 'failure'
            
        

In [116]:
res = solve(board)
print(res)
print(board)

{'A1': 4, 'A2': 8, 'A3': 3, 'A4': 9, 'A5': 2, 'A6': 1, 'A7': 6, 'A8': 5, 'A9': 7, 'B1': 9, 'B2': 6, 'B3': 7, 'B4': 3, 'B5': 4, 'B6': 5, 'B7': 8, 'B8': 2, 'B9': 1, 'C1': 2, 'C2': 5, 'C3': 1, 'C4': 8, 'C5': 7, 'C6': 6, 'C7': 4, 'C8': 9, 'C9': 3, 'D1': 5, 'D2': 4, 'D3': 8, 'D4': 1, 'D5': 3, 'D6': 2, 'D7': 9, 'D8': 7, 'D9': 6, 'E1': 7, 'E2': 2, 'E3': 9, 'E4': 5, 'E5': 6, 'E6': 4, 'E7': 1, 'E8': 3, 'E9': 8, 'F1': 1, 'F2': 3, 'F3': 6, 'F4': 7, 'F5': 9, 'F6': 8, 'F7': 2, 'F8': 4, 'F9': 5, 'G1': 3, 'G2': 7, 'G3': 2, 'G4': 6, 'G5': 8, 'G6': 9, 'G7': 5, 'G8': 1, 'G9': 4, 'H1': 8, 'H2': 1, 'H3': 4, 'H4': 2, 'H5': 5, 'H6': 3, 'H7': 7, 'H8': 6, 'H9': 9, 'I1': 6, 'I2': 9, 'I3': 5, 'I4': 4, 'I5': 1, 'I6': 7, 'I7': 3, 'I8': 8, 'I9': 2}
{'A1': 0, 'A2': 0, 'A3': 3, 'A4': 0, 'A5': 2, 'A6': 0, 'A7': 6, 'A8': 0, 'A9': 0, 'B1': 9, 'B2': 0, 'B3': 0, 'B4': 3, 'B5': 0, 'B6': 5, 'B7': 0, 'B8': 0, 'B9': 1, 'C1': 0, 'C2': 0, 'C3': 1, 'C4': 8, 'C5': 0, 'C6': 6, 'C7': 4, 'C8': 0, 'C9': 0, 'D1': 0, 'D2': 0, 'D3': 8,

In [117]:
def show_board(inp):
    print(np.array(list(inp.values())).reshape((9,9)))

In [118]:
show_board(res)

[[4 8 3 9 2 1 6 5 7]
 [9 6 7 3 4 5 8 2 1]
 [2 5 1 8 7 6 4 9 3]
 [5 4 8 1 3 2 9 7 6]
 [7 2 9 5 6 4 1 3 8]
 [1 3 6 7 9 8 2 4 5]
 [3 7 2 6 8 9 5 1 4]
 [8 1 4 2 5 3 7 6 9]
 [6 9 5 4 1 7 3 8 2]]


In [119]:
coord = 'E5'

def all_legal_vals(coord, board):
    legal_vals = []
    for n in range(1,10):
        if is_legal(n, coord, board):
            legal_vals.append(n)
    return legal_vals

In [127]:
def mrv(board):
    vals = [(all_legal_vals(coord, board), coord) for coord in board.keys()]
    vals = [i for i in vals if len(i[0])>0]
    vals.sort(key = lambda x : len(x[0]))
    if vals:
        mrv_coord = vals[0][1]
        mrv_vals = vals[0][0]
        return (mrv_coord, mrv_vals)

In [128]:
mrv(board)

('B4', [4])

In [136]:
calls = 0
def solve_mrv(board):
    global calls
    calls += 1
    b = board.copy()
    if not 0 in b.values():
        return b
    coord, vals = mrv(b)
    print(calls, coord, vals)
    for n in vals:
        b[coord] = n      
        result = solve(b)
        if result != 'failure':
            return result
        b[coord] = 0
    return 'failure'
            

In [137]:
solve_mrv(board)

1 B4 [4]


'failure'